In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from time import sleep

In [3]:
from importlib import reload

In [4]:
import drivers, lib
from time import sleep
import pickle

In [5]:
from lib import data_management as dm

In [6]:
# reload(drivers.IQAWG)

In [7]:
# reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
from drivers.Agilent_PNA_L import *
from drivers.znb import *
# reload(drivers.KeysightAWG)
from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.Tektronix_AWG5014 import *
from drivers.IQAWG import *
from drivers.E8257D import MXG, EXG

In [8]:
exa = Agilent_EXA_N9010A("EXA")
# ro_awg = KeysightAWG("AWG3")
# q_awg = KeysightAWG("AWG2")
# ro_lo = EXG("PSG")

In [9]:
# awg._visainstrument.close()
awg3 = KeysightAWG("AWG3")
awg2 = KeysightAWG("AWG2")
ro_awg = IQAWG(AWGChannel(awg3, 1), AWGChannel(awg3, 2))
q_awg = IQAWG(AWGChannel(awg2, 1), AWGChannel(awg2, 2))

In [10]:
q_lo = MXG("MXG")

In [11]:
ro_lo = Agilent_PNA_L("PNA-L1")
ro_lo.set_frequency = ro_lo.set_center
ro_lo.set_output_state = lambda x: x
ro_lo.set_span(0)
ro_lo.set_nop(1)
ro_lo.sweep_continuous()

In [9]:
ro_lo = Znb("ZNB")
ro_lo.set_frequency = ro_lo.set_center
ro_lo.set_output_state = lambda x: x
ro_lo.set_nop(1)
ro_lo.sweep_continuous()

In [14]:
# reload(lib.iq_mixer_calibration)
from lib.iq_mixer_calibration import *

##  Readout resonator

In [91]:
ro_resonator_frequency = 7.535e9
if_offset = 0

In [92]:
cal = IQCalibrator(ro_awg, exa, ro_lo, "Xmons_Nb_bandage_res_mixer_5", -10)
res = cal.calibrate(lo_frequency=ro_resonator_frequency, if_frequency=if_offset, lo_power=7,
                    ssb_power=-35, waveform_resolution=1, iterations=7, minimize_iterlimit=20,sa_res_bandwidth=200,)
#                     initial_guess=res.get_optimization_results()[0])
#                     initial_guess = {'dc_offsets': array([ 0.1,  .1]),
#                                     'dc_offsets_open': array([ 1  ,  1]),
#                                     'if_amplitudes': None,
#                                     'if_offsets': None,
#                                     'if_phase': None})
print(res)

Calibration data for mixer Xmons_Nb_bandage_res_mixer_5
Mixer parameters: {'mixer_id': 'Xmons_Nb_bandage_res_mixer_5', 'iq_attenuation': -10}
Radiation parameters: {'lo_frequency': 7535000000.0, 'lo_power': 7, 'if_frequency': 0, 'ssb_power': -35, 'waveform_resolution': 1}
Optimization results: {'dc': -102.40679168701172, 'dc_open': [-35.00094985961914]}
Optimization parameters {'dc_offsets': array([ 0.02173984, -0.05006656]), 'dc_offsets_open': array([ 1.15698962,  1.15713221]), 'if_offsets': None, 'if_amplitudes': None, 'if_phase': None}
Optimization time: 0 h 0 m 26.23 s
Finished at: 2017-08-18 10:09:25.130742


In [93]:
dm.save_IQMX_calibration(res)

In [15]:
calibrations=dm.load_IQMX_calibration_database("Xmons_Nb_bandage_res_mixer_to_-10", -10)
res = calibrations.get(frozenset(dict(lo_power=10, ssb_power=-10, lo_frequency=7.535e9, 
                                      if_frequency=50e6, waveform_resolution=1).items()))

In [16]:
calibrations

{frozenset({('if_frequency', 0),
            ('lo_frequency', 7535100000.0),
            ('lo_power', 7),
            ('ssb_power', -10),
            ('waveform_resolution',
             1)}): <lib.iq_mixer_calibration.IQCalibrationData at 0xa19b0b8>}

## Qubit

In [41]:
qubit_frequency = 6.163e9
if_frequency = 100e6

In [51]:
cal = IQCalibrator(q_awg, exa, q_lo, "Xmons_Nb_bandage_qub_mixer_5", -6)
res = cal.calibrate(lo_frequency=qubit_frequency+if_frequency, if_frequency=if_frequency, lo_power=8,
                    ssb_power=-17, waveform_resolution=.5, iterations=5, minimize_iterlimit=30,sa_res_bandwidth=100)
      #          initial_guess=res01.get_optimization_results()[0])
print(res)

Calibration data for mixer Xmons_Nb_bandage_qub_mixer_5
Mixer parameters: {'mixer_id': 'Xmons_Nb_bandage_qub_mixer_5', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 6263000000.0, 'lo_power': 8, 'if_frequency': 100000000.0, 'ssb_power': -17, 'waveform_resolution': 0.5}
Optimization results: {'dc': -77.934364318847656, 'if': [-16.656984329223633, -46.59967041015625, -122.78775024414062]}
Optimization parameters {'dc_offsets': array([ 0.00379474, -0.016923  ]), 'dc_offsets_open': None, 'if_offsets': array([ 0.00398447, -0.02057001]), 'if_amplitudes': array([ 0.44026808,  0.46053226]), 'if_phase': array([-1.49267587])}
Optimization time: 0 h 2 m 34.77 s
Finished at: 2017-08-22 11:57:42.644058


In [38]:
res01 = res

In [26]:
res05m10=res

In [52]:
dm.save_IQMX_calibration(res)

In [50]:
q_lo._visainstrument.close()

In [31]:
awg._visainstrument.close()

In [ ]:
q_awg.get_1st_delay()

In [ ]:
exa.set_singlesweep_mode()

In [ ]:
exa.set_centerfreq(6e9)

In [ ]:
data_u = exa.make_sweep_get_data()
freqs_u = exa.get_freqpoints()

In [ ]:
plt.plot(freqs_u,data_u)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(6.5, 4.5)
plt.annotate('$P_{LO}= 7$ dBm\n'+'$P_{SB}=-15$ dBm\n'+'Uncalibrated', xy=(0.7, 0.75), xycoords='axes fraction', fontsize=16)
plt.grid()
plt.savefig('./cal_pictures/uncal2.pdf', dpi=300)

In [ ]:
q_awg.output_continuous_wave(frequency=50e6, amplitude=1, phase=0, offset=0,
                                 waveform_resolution=1, channel=1)
q_awg.output_continuous_wave(frequency=50e6, amplitude=1, phase=0, offset=0,
                                 waveform_resolution=1, channel=2)

In [ ]:
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['xtick.labelsize'] = 16
matplotlib.rcParams['ytick.labelsize'] = 16

In [ ]:
q_lo.set_power(7)

In [ ]:
l = [1,2,3]
d = [4,5,6]

In [ ]:
l+[len(d)]

In [ ]:
for i,j in zip(l,d):
    print(i,j)

In [ ]:
import numpy as np

In [ ]:
np.product([[1,1,1],[1,1,1]])